# Workshop 9

This workshop includes three exercises
- a visualization of the negative log likelihood estimation
- an introduction to the profiled log likelihood ratio
- an exercise with data sample to be used for the final project

The first and second exercises will be helpful for your homework 4. The last one will be useful for your final project.

**Submit your completed notebook and the pdf version of it to the bcourse to receive credit. Please rename your files so that they start with your SID.**

## Submission Deadline: 13:59 pm November 4, 2022

# 1. Visualization of the negative log likelihood estimation

This exercise consists of the following components:
- data sample:
    - the data sample is generated from a Gaussian PDF with a mean of 125 and a standard deviation of 2
    - the entries of this sample are binned in the range of (100,160) with 60 bins and shown in a histogram; in other words, the sample include 60 independent measurements
- construct the binned negative log likelihood function
    - Histogram the data sample to create binned observed data
    - Expectation: calculated from the normal distribution with a mean of 125 and a standard deviation of 2 at the bin centers of the histogram
        - recall `PDF(x)dx` gives the probability of observing the outcome in the interval of $dx$
        - so at a given point of $x$, the expectation is $x\cdot dx$, where $dx$ is taken to be the bin width
    - Calculate the negative log Poisson terms of the 60 independent measurements and sum them up

In [ ]:
%notebook inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML
from ipywidgets import interact, interactive, fixed, interact_manual
from matplotlib import animation, rc
#sns.set_context('poster')
from scipy.stats import poisson
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.stats import chi2

# Import all the packages needed

In [ ]:
#Write code to generate 50 random numbers
# from a normal distribution with a mean of 125
# and a standard deviation of 2



In [ ]:
#Histogram the sample
# Use 30 bins and a range of (110,140)
# Get the bin centers as a numpy array
# Get the bin counts (the number of entries in each bin) as a numpy array
datacount, binedges = np.histogram(data,bins=30,range=(110,140))
bincenters = 0.5*(binedges[:-1]+binedges[1:])


- Develop the negative log likelihood function here
- **Read the comments very carefully**

In [ ]:
def NLL(x,datacount, bincenters):
    # the function has four input arguments
    # x is a list, and its elements are [mean, sigma] of the normal distribution
    # datacount is the bin count numpy array
    # bincenters is the np array of bincenters
    # binwidth is a scalar, given by (max-min)/nbins
    

    # norm.pdf(bincenters,x[0],x[1]) gives the PDF values at a series of points
    # binwidth is the interval ("dx")
    binwidth = bincenters[1]-bincenters[0]
    expectation_without_normalization = norm.pdf(bincenters,x[0],x[1])*binwidth

    # the sum of expectation_without_normalization should be 1 
    # because the total probability is 1
    # the line below gives the expectation normalized to number of entries in the sample
    expectation = expectation_without_normalization*datacount.sum()

    # three important elements below
    # negative => -1*
    # log likelihood => log.pmf
    # sum. you should sum up all the individual measurement terms ==> .sum()
    return -1*poisson.logpmf(datacount,expectation).sum()

Minimize the negative log likelihood value

In [ ]:
result = minimize(NLL, # objective function
                  [126,1], # initial values for the free parameters 
                  args=(datacount, bincenters), # other input arguments to the minimization
                  method='Nelder-Mead' # minimization method
                 )

In [ ]:
# Print the fit result
print(result) 

#Retrive information from the fit results
mean,std = result.x
NLLvalue = result.fun

## Visualization 

Complete the code cell below
**example output plot**
<img src="https://portal.nersc.gov/project/m3438/physics77/WS09/fig1.png" width='500'>

In [ ]:
plt.hist(data,bins=30,range=(110,140), label='Generated random data')

# Recycle the code used earlier to get the 
# expectation 
# Note that the mean and std parameters of the normal
# function are now from the fit result

# Add the missing line back

expectation_without_normalization = norm.pdf(bincenters,result.x[0],result.x[1])


plt.plot(bincenters,expectation,'r--',label='Fitted Normal distribution')
plt.xlim(120,130)
plt.ylim(0, np.max(datacount)*1.5)
plt.legend(loc='upper right', fontsize=14)

- when we define the NLL function, we need to calculate the expectation from the PDF, and we need to normalize the expectation to that of the observation. This was done by the line `expectation = expectation_without_normalization*datacount.sum()`
- we will define a new NLL function, named as `NLL_v2`, where we leave the normalization of the expectation to be a free parameter, and we let the minimization (fit) decide it.

In [ ]:
# Find how this cell differs from the cell where NLL was defined
def NLL_v2(x,datacount, bincenters):
    # the function has four input arguments
    # x is a list, and its elements are [mean, sigma] of the normal distribution
    # datacount is the bin count numpy array
    # bincenters is the np array of bincenters
    # binwidth is a scalar, given by (max-min)/nbins
    

    # norm.pdf(bincenters,x[0],x[1]) gives the PDF values at a series of points
    # binwidth is the interval ("dx")
    binwidth = bincenters[1]-bincenters[0]
    expectation_without_normalization = norm.pdf(bincenters,x[0],x[1])*binwidth

    # the sum of expectation_without_normalization should be 1 
    # because the total probability is 1
    # the line below gives the expectation normalized to number of entries in the sample
    expectation = expectation_without_normalization*x[2]

    # three important elements below
    # negative => -1*
    # log likelihood => log.pmf
    # sum. you should sum up all the individual measurement terms ==> .sum()
    return -1*poisson.logpmf(datacount,expectation).sum()

In [ ]:
result_v2 = minimize(NLL_v2, # objective function
                  [126,1,10], # initial values for the free parameters 
                  args=(datacount, bincenters), # other input arguments to the minimization
                  method='Nelder-Mead' # minimization method
                 )

In [ ]:
# Print the fit result
print(result_v2) 

#Retrive information from the fit results
mean,std, normalizationfactor = result_v2.x
NLLvalue = result_v2.fun

What is the value of x[2] here? What value should it be?

## Visualize
- in the cell blow, we use ipywidgets package to create a control that allows you to tune the mean value and the standard deviation value (std) of the Normal distribution "by hand". 
- the initial values of these parameters are taken from the minimization (fit) results 
- vary the value of mean, and see
    - on the top panel, how the expected normal distribution shifts
    - on the bottom panel, how the negative log likelihood value varies
- **in the markdown cell below, type the measured central value as well as it uncertainties**. You can tell these by varying the mean value and see how the NLL changes

In [ ]:
@interact(mean=widgets.FloatSlider(min=124.0, max=125.0, step=0.001, value=result.x[0],layout=widgets.Layout(width='40%') , readout_format='.3f'), std=widgets.FloatSlider(min=0.1, max=3.141, step=0.1, value=result.x[1]), continuous_update=True)
def draw_plt(mean, std):
    fig, axs = plt.subplots(3,1,figsize=(16,9), gridspec_kw={'height_ratios': [3, 1, 2]})
    plt.subplot(311)
    plt.hist(data,bins=30,range=(110,140))



    reference = norm.pdf(bincenters,125,2)*binwidth*datacount.sum()
    expectation = norm.pdf(bincenters,mean,std)*binwidth*datacount.sum()
    plt.plot(bincenters,expectation,'r--',label="Expectation")
    plt.plot(bincenters,reference,'g--',label="Reference")
    plt.xlim(120,130)
    plt.ylim(0,1.2*datacount.max())
    plt.legend()
    plt.ylabel('Number of entries',fontsize=14)
    
    # Plot the residual
    plt.subplot(312)
    residual = datacount - expectation 
    plt.errorbar(bincenters, residual, yerr=np.sqrt(datacount), fmt='o',color='black')
    plt.xlim(120,130)
    plt.ylabel('Residual: data - fit',fontsize=14)
    
    
    
    plt.subplot(313)

    mass = np.linspace(110,140,3001)
    NLLval=np.ones(0)
    for m in mass:
        NLLval = np.hstack( (NLLval,  NLL_v2([m,std,50], datacount,bincenters)))

    NLLmin = NLLval.min()    
    NLLval = NLLval - NLLmin
    
    NLL_parameters = NLL_v2([mean,std, 50], datacount,bincenters ) - NLLmin
    
    plt.plot(mass,NLLval)
    plt.scatter(mean, NLL_parameters, color="red",label='Current NLL value')
    # print(mean, NLL_parameters)
    plt.ylim(0,4)
    plt.ylabel('NLL',fontsize=14)
    plt.xlim(result.x[0]-0.5,result.x[0]+0.5)
    plt.plot([123.,126.5],[0.5,0.5],'r--')
    plt.legend()

# Type down the $\pm 1 \sigma$ uncertainty in this cell 


# Mean = 1xx.xx $^{+ yy}_{-zz}$

for example, 127.32$^{+0.91}_{-0.32}$

# 2. Hypothesis testing and profile likelihood ratio

- in this exercise, we define two different hypotheses, namely, the b-only hypothesis and the signal-plus-background hypothesis
- then, we generate one data set, which is considered as the observed data
- we will test this observed data set against the B-only hypothesis and determine the p-value of the observed data set

In [ ]:
# in this experiment, there are 50 independent observations
# the b-only expectation is given by the line below
B = np.arange(50,100)

# the signal expectation is give by 
S = norm.pdf(np.linspace(50,100,50),75,2)*50


# The signal-plus-background expectation is given by
SB = B + S

# Observed data is given by
obs = np.array([51,49,44,58,48,68,62,49,61,61,83,52,64,82,66,78,70,67,60,78,74,72,69,77,81,92,85,84,66,77,75,81,81,96,89,85,72,77,76,106,95,104,110,99,113,91,93,77,100,79])

**Complete the code cell below to visualize the data and hypotheses**

**Example output**
<img src="https://portal.nersc.gov/project/m3438/physics77/WS09/fig2.png" width=500>

In [ ]:
#visualize the hypotheses and the observed data 

plt.subplots(figsize=(8,6))
bincount, binedges, others=plt.hist(obs[0],bins=50,range=(50,100))
bincenters = (binedges[1:]+binedges[0:-1])*0.5
plt.hist(bincenters, bins=50, range=(50,100),weights=S+B,histtype="step",ls='dashed',color='red',label="Expected signal")
plt.hist(bincenters, bins=50, range=(50,100),weights=B,color='green',label="Background")
plt.errorbar(bincenters, obs, yerr=np.sqrt(obs), fmt='o',color='black')

plt.ylabel("Number of events")
plt.xlim(50,100)
plt.legend(frameon=False)

## Define the negative log likelihood function 

## $$
  NLL(\mu) = \sum_i{ -\log{Pois}(d_i| \mu s_i + b_i)}
$$

where $i$ is the index of the bin. {$s_i$} and {$b_i$} are given by the numpy arrays S and B, respectively, $\mu$ is the signal strength parameter that scales up and down the signal expectation, and {$d_i$} is the experimental outcome from either real data or pseudo experiments. When $\mu$ = 0, this expression becomes the negative log likelihood for the b-only hypothesis. When $\mu$ = 1, this expression gives the negative log likelihood for the signal-plus-background hypothesis.

**Reshape arrays to have 2 dimensions**

In [ ]:
# These arrays will now have 2 dimensions
# the values are on the axis 1
# there is one entry on axis 0
# This will make it easier for caluclations that involve pseudo experiments
# for which PE data are saved as arrays of shape (N,50)
# where N is the number of pseudo experiments
S=S.reshape(1,50)
B=B.reshape(1,50)
obs=obs.reshape(1,50)

In [ ]:
# Define this function for scipy.optimize.minimize
# the first argument must be the free parameter
# that's determined by the minimization
# the next three are the S, B, and observed data
def NLLfunc(x,S,B,obs):
    exp = x[0]*S+B
    # the second argument, axis = 1, implies that
    # these arrays (obs, exp) have more than one axes
    return np.sum(-poisson.logpmf(obs,exp),axis=1)

In [ ]:
# Generate 10,000 Pseudo experiments from b-only hypothesis
BPE = rng.poisson(B,size=(10000,50))

## Test statistic 1: Negative Log Likelihood Ratio

## $$NLLR = -2\log\frac{L(data|s+b)}{L(data|b-only)}
$$

- calculate the NLLR for each PE
- calculate the observed NLLR (i.e., the NLLR value for the observed data)

In [ ]:
# NLLR values for the PEs
# These can be done with three simple lines 
NLLSB = NLLfunc([1], S, B, BPE)
NLLB = NLLfunc([0], S, B, BPE)
NLLR = 2*(NLLSB - NLLB)

In [ ]:
# The NLLR value for the observed data
NLLSB_obs = NLLfunc([1], S, B, obs)
NLLB_obs = NLLfunc([0], S, B, obs)
NLLR_obs = 2*(NLLSB_obs - NLLB_obs)

**Complete the cell below to visualize the hypothesis testing with P.E.s**

**Example output**

<img src="https://portal.nersc.gov/project/m3438/physics77/WS09/fig3.png" width=500>

In [ ]:
# Visualize the PE based hypothesis testing

# fix the lines below
plt.hist(,bins=50,range=(-20,20),label='B-only P.E.s')
plt.hist(,bins=50,range=(-20,20),color='magenta',label='Tail NLLR < NLLR$_{obs}$')
plt.plot([],[0, 300],label='Observed LLR')
plt.ylabel('Number of Entries')
plt.xlabel('Negative Log Likelihood Ratio (NLLR)')
plt.legend(fontsize=12)

- The magneta area gives the fraction of PE outcomes that are more extreme than the observed data outcome
- the p-value is the fraction of outcomes corresponding to the magenta area
- the significance can be converted from the p-value using Z = norm.ppf(1-pvalue)

In [ ]:
# Develop your code to tell us the p-value and the significance of rejecting the background only hypothesis

pvalue=
Z = 
print('p-value of the background-only hypothesis is {:1.4f}'.format(pvalue))
print('the Statistical significance of rejecting the background only hypothesis is {:1.2f}'.format(Z))

## Test statistic 2: Profile Likelihood Ratio

## $$PLR = -2\log\frac{L(data|b-only)}{L(data|\hat{\mu}S+B)}
$$
The numerator is the likelihood constructed from data and the background only hypothesis, and the denominator is the likelihood constructed from data and the $\mu$ times Signal-plus-background hypothesis, where the signal strength parameter is determined by minimizing the negative log likelihood hood.

- unlike the first test statistic, the denominator of the PLR requires a minimization of the negative log likelihood
- the only free parameter in this example is $\mu$; varying the $\mu$ value will lead to a minimized negative log likelihood function
- **The hypothesis that appears at the numerator is the one that we test**
- **The free parameter that appears at the denominator is the parameter of interest**, which defines the hypothesis.
- **The PoI in the numerator is fixed to the value of the hypothesis.** In this case, b-only hypothesis requries $\mu$ = 0

**First, as an exercise, let's develop the code to calculate the PLR value for the observed data**

In [ ]:
# Calculate the PLR for the observed data

# Minimize the negative log likelihood function created earlier
# only one free parameter 
result_obs = minimize(NLLfunc,[1], bounds=[(0,10)],args=(S,B,obs))

# the minimization result would return the minimized objective function
NLLmufree_obs = result_obs.fun

# NLLfunc([0], S, B, obs) gives the numerator of the PLR
PLR_obs = 2*(NLLfunc([0], S, B, obs) -  NLLmufree_obs)

print(PLR_obs)

**Now, let's calculate the PLR values for the b-only pseudo experiments that we generated earlier**

In [ ]:
# we will have to do a for-loop here

# all the PLR values from the P.E.s will be stored in the numpy array PLR
PLR = np.ones(0)

# all the fitted mu values from the P.E.s will be stored in the numpy array Fitted_mu 
Fitted_mu = np.ones(0)

for PE in BPE:
    # if you are not familiar with this 
    # check out the shape of PE and BPE here
    
    # Write your code to calculate the PLR for this current PE
    # Just follow the example given to you in the previous cell
    result = minimize(     args=(S,B,PE))
    NLLmufree = 
    NLLB = 
    PLR_PE = 

    
    PLR = np.hstack( (PLR, PLR_PE))
    Fitted_mu = np.hstack( (Fitted_mu, result.x[0]))
    

**Complete the cell below to visualize the hypothesis testing with P.E.s**

**Example output**

<img src="https://portal.nersc.gov/project/m3438/physics77/WS09/fig4.png" width=500>

In [ ]:
# Fix this cell

plt.hist(,bins=25,range=(0,25),label='B-only P.E.s')

plt.hist(,bins=25,range=(0,25),color='magenta',label='tail (PLR > PLR$_obs$)')
# Note that the magenta entry on the left of the vertical line is a binning effect
# The bin edges and the observed PLR value are not perfectly aligned


plt.yscale('log')
plt.plot([,[0,1000], label='Observed PLR')

# Generating a chi-2 distribution as a reference
chi2_curve = chi2.pdf(np.linspace(0.5,24.5,25),1)*0.5*PLR.size
plt.plot(np.linspace(0.5,24.5,25), chi2_curve,label='chi-2 distribution')

plt.legend(fontsize=14)



### Now, use the profile likelihood ratio based test statistic to calculate the p-value significance
#### from pseudo experiments
- the p-value is the fraction of outcomes corresponding to the magenta area
- the significance can be converted from the p-value using Z = norm.ppf(1-pvalue)

In [ ]:
pvalue = 
Z = 
print('p-value of the background-only hypothesis is {:1.4f}'.format(pvalue))
print('the Statistical significance of rejecting the background only hypothesis is {:1.2f}'.format(Z))

### analytically
We introduced the PLR because its distribution is known. If the hypothesis to test (the one that appears at the numerator of the PLR) and the hypothesis used to generate the pseudo experiments are the same, then the PLR distribtuion of the P.E.s is a chi-squared distribution, which is illustrated in the figure above.

-**We also know that the observed significance is given by** 
## $$Z = \sqrt{PLR_{obs}}$$

- now use this relation to calculate the significance, and then convert the significance to the p-value. Do they agree with your earlier calculation using the NLLR?
- The advantage of using the PLR is that we do not have to generate a large number of P.E.s when the observed significance is large. We only need to calculate the PLR$_obs$, and its square-root gives the observed signfiicance

In [ ]:
Z = 
# This is how you get pvalue, knowing Z
pvalue = 1-norm.cdf(Z)
print('p-value of the background-only hypothesis is {:1.4f}'.format(pvalue[0]))
print('the Statistical significance of rejecting the background only hypothesis is {:1.2f}'.format(Z[0]))

# 3. Final Project Data

- Data for the final project are saved in a h5 file. We will show you how to get this h5 file and how to retrieve the data numpy array
- We will also use this data set to do some binned maximum log likelihood fit exercise

In [ ]:
import numpy as np
import h5py

In [ ]:
# Downloading this h5 file from the web
import os
os.system("wget https://portal.nersc.gov/project/m3438/physics77/data/datalhc.h5")

In [ ]:
# Open h5 file
h = h5py.File("datalhc.h5",'r')

# Retrieve the data array 
data = h["dataset"][:]

# Check its shape
print(data.shape)

The shape of data array is (1178902, 10), indicating that there are 1,178,902 collision events that contain two photons. The axis 1 has 10 entries. They are

- transverse momentum of photon 1
- pseudo rapidity of photon 1
- azimuthal angle of photon 1
- energy of photon 1
- transverse momentum of photon 2
- pseudo rapidity of photon 2
- azimuthal angle of photon 2
- energy of photon 2
- Event Number, which is an index of the collision event
- Run Number, which is an index of a `run`. At LHC, the detector is often run for an extended period of time, raning from a few minuts to a few hours. Data events collected in the same data taking period are said to be in the same `run`.

In [ ]:
# Here a few functions are defined to get the px, py, pz components of the momentum

def px(pt, phi):
    return pt*np.cos(phi)

def py(pt,phi):
    return pt*np.sin(phi)

def pz(pt, eta):
    return pt*np.sinh(eta)


# Using energy and momentum we can calculate the mass of a particle or a multi-particle system
def mass(E,px,py,pz):
    return np.sqrt(E**2 - (px**2+py**2+pz**2))


In [ ]:
# We will use the functions defined above to calculate 
# the px, py, pz components of the diphoton momentum
px_yy = 
py_yy =
pz_yy = 

# We will also calculate the energy of the diphoton system
# which is the sum of individual photons
E_yy = 

# Finally, we can calculate the diphoton mass
m_yy = mass(E_yy, px_yy,py_yy,pz_yy)

## Plot the $m_{\gamma\gamma}$ distribution
- a tiny bump around 125 GeV is already discernible 

In [ ]:
# How does the diphoton mass distribution look like?

obs, binedges,others =plt.hist(m_yy,bins=55,range=(105,160),label='Data 2015-2018')
plt.xlabel('$m_{\gamma\gamma}$ [GeV]')
plt.ylabel('Number of entries')
plt.legend()

## Use the maximum log likelihood method to fit a 4th order polynomial to data
- define a 4th order polynomial function, `poly4`
- define the NLL for this `poly4`. 
    - For the sake of simplicity, we calculate the value of the polyminomial at np.linspace(105.5,159.5,55), i.e., the bin centers of the above histogram. The binwidth is 1
- Use scipy.optimize.minimize to perform the fit, in which the coefficients of the terms are determined
- Visualize the fitted Polynomial

In [ ]:
# define a fourth-order polynomial
def poly4(myy,c):
    return 1 + c[0]*myy + c[1]*myy**2 + c[2]*myy**3 + c[3]*myy**4

In [ ]:
def NLLpoly(c, obs):
    # Create a sequence of values at which the expectation will be evaluated
    myy = np.linspace(105.5,159.5,55)
    # The expectaiton. Note that the c[4] factor will take care of the normalization of the polynomial PDF
    exp = poly4(myy, [c[0],c[1],c[2],c[3]])*c[4]
    # return a negative log likelihood
    # again, needs to sum over all 55 bins
    NLLvalue = -1*poisson.logpmf(obs,exp).sum()
    return NLLvalue

### Fit the 4th order polynomial to data
- print out the fit result
- visualize the fitted function and data distribution
**Does the fit make sense?**

In [ ]:
result = minimize(NLLpoly,x0=[-10,100,1,1,1],args=(obs),method='Nelder-Mead')
print(result)

# Pass the fitted values back to the list c
c=result.x

In [ ]:
# Use the fitted coefficients to create the expectation
myy = np.linspace(105.5,159.5,55)
fit = poly4(myy, [c[0],c[1],c[2],c[3]])*c[4]

# Data
obs, binedges,others =plt.hist(m_yy,bins=55,range=(105,160), label='data')
plt.plot(myy,fit,'r--', label='Fitted Polynomial')
plt.legend(fontsize=14)
plt.xlabel('$m_{\gamma\gamma}$ [GeV]')
plt.ylabel('Number of entries')

## Execute the next cell repeatedly
- the minimization algorithm is pretty rudimentary. It doesn't perform well. 
- the first fit is unlikely to be a good one
- in the next cell, we pass the fit result to c, and use them as the initial value in a new iteration of minimization
- repeatedly execute the next cell and see if the NLL decreases and if the fit continues to improve

**Your final fit plot should look like **

<img src="https://portal.nersc.gov/project/m3438/physics77/WS09/fig6.png" width=500>

In [ ]:
result = minimize(NLLpoly,x0=c,args=(obs),method='Nelder-Mead')
print('Pay attention to the NLL value {:6.3f}'.format(result.fun))
c=result.x
fit = poly4(myy, [c[0],c[1],c[2],c[3]])*c[4] 
obs, binedges,others =plt.hist(m_yy,bins=55,range=(105,160), label='data')
plt.plot(myy,fit,'r--', label='Fitted Polynomial')
plt.legend(fontsize=14)
plt.xlabel('$m_{\gamma\gamma}$ [GeV]')
plt.ylabel('Number of entries')

In [ ]:
print("The minimized NLL value with bkg-only PDF is {:5.2f}".format(   ))

## Fit data with a signal plus background PDF
- In this exercise, we will create a different function
$$
f(x=m_{\gamma\gamma}) = n_b \sum_i c_i x^i + n_s \mathrm{Normal}(x,125,1.6)
$$
    - in this setup, the first term is a fourth order polynomial, and its normalized to $n_b$ which is the number of background events. The second term is a normal distribution with a mean of 125 and a standard deviation of 1.6, and its normalization $n_s$ represents the number of signal events. 
    - the shape (mean and std. dev.) of the Gaussian distribution is fixed 
    - the free parameters in the fit include
        - the coefficients of the polynomial
        - the normalizations of the signal and background PDFs ($n_s$ and $n_b$)

In [ ]:
# define the signal pdf

def sigpdf(myy,mean,std):
    return norm.pdf(myy,mean,std)

In [ ]:
# Define negative log likelihood value with the observed data and the signal plus background PDF
def NLLSpluspoly(c, obs):
    myy = np.linspace(105.5,159.5,55)
    exp = poly4(myy, [c[0],c[1],c[2],c[3]])*c[4] + sigpdf(myy,125,1.6)*c[5]
    NLLvalue = -1*poisson.logpmf(obs,exp).sum()
    return NLLvalue

Now we perform an initial fit to the observed data

In [ ]:
# Note that I am using the coefficients determined from the previous fits as the
# initial values here
result = minimize(NLLSpluspoly,x0=np.hstack((c,1000)),args=(obs),method='Nelder-Mead')
c1=result.x
print('Pay attention to the NLL value {:6.3f}'.format(result.fun))
myy = np.linspace(105.5,159.5,55)
fit = poly4(myy, [c1[0],c1[1],c1[2],c1[3]])*c1[4] + sigpdf(myy,125,1.6)*c1[5]
# Data
obs, binedges,others =plt.hist(m_yy,bins=55,range=(105,160), label='data')
plt.plot(myy,fit,'r--', label='Fitted Polynomial')
plt.legend(fontsize=14)
plt.xlabel('$m_{\gamma\gamma}$ [GeV]')
plt.ylabel('Number of entries')


### Execute the cell below repeatedly to see if fit gets improved?

**Final output**

<img src="https://portal.nersc.gov/project/m3438/physics77/WS09/fig7.png" width=500>

In [ ]:
result = minimize(NLLSpluspoly,x0=c1,args=(obs),method='Nelder-Mead')
c1=result.x
print('Pay attention to the NLL value {:6.3f}'.format(result.fun))
myy = np.linspace(105.5,159.5,55)
fit = poly4(myy, [c1[0],c1[1],c1[2],c1[3]])*c1[4] + sigpdf(myy,125,1.6)*c1[5]
# Data
obs, binedges,others =plt.hist(m_yy,bins=55,range=(105,160), label='data')
plt.plot(myy,fit,'r--', label='Fitted Polynomial')
plt.legend(fontsize=14)
plt.xlabel('$m_{\gamma\gamma}$ [GeV]')
plt.ylabel('Number of entries')


In [ ]:
print('The fitted number of signal events is {:4.2f} '.format(   ))

print("The minimized NLL value with signal-plus-bkg PDF is {:5.2f}".format(   ))

## Can you develop the code to fit the data with a signal plus background pdf where the mean and sigma of the signal pdf are also free?

In [ ]:
def NLLSpluspolyfree(c, obs):

    # complete the cell here
    
    return NLLvalue

In [ ]:
# Your fit code

In [ ]:
# Your fit code

In [ ]:
print('The fitted mean value of the signal is {:4.2f} GeV'.format(  ))
print('The fitted sigma value of the signal is {:4.2f} GeV'.format( ))
print('The fitted number of signal events is {:4.2f} '.format(  ))

Congratulations for completing this workshop!